In [1]:
try:
    import qiskit
    import qiskit_superstaq as qss
except ImportError:
    print("Installing qiskit-superstaq...")
    %pip install -q qiskit-superstaq[examples]
    print("Installed qiskit-superstaq. Please restart the kernel to import the packages.")
from general_superstaq import ResourceEstimate
from qiskit.circuit.random import random_circuit

# Getting Resource Estimate Costs

In [2]:
# Connect to Superstaq
# Provide your api key to the using the "api_key" argument if
# SUPERSTAQ_API_KEY environment variable is not set.
superstaq = qss.superstaq_provider.SuperstaqProvider()

# Resource Estimate for Simple Circuit

In [3]:
# Create simple circuit and submit to simulated device
circuit = qiskit.QuantumCircuit(2)
circuit.cx(0, 1)
circuit.h(1)

circuit.draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

In [4]:
resource_estimate = superstaq.resource_estimate(circuit, "ss_unconstrained_simulator")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=1, num_two_qubit_gates=1, depth=2)


# Resource Estimate for More Complex Random Circuit

In [5]:
# Create and submit random circuit to simulated device
circuit_random = random_circuit(4, 4, measure=True)
circuit_random = qiskit.transpile(circuit_random, basis_gates=["id", "u1", "u2", "u3", "cx"])
circuit_random.draw(fold=-1)

global phase: 4.6777
     ┌────────────┐                                                                              ┌─────────────┐     ┌───┐┌───────────────────┐┌───┐┌────────────┐                   ┌─┐   
q_0: ┤ U1(4.5005) ├─────────────────────────────────────■────────────────────────────────■───────┤ U1(-2.2713) ├─────┤ X ├┤ U3(-0.016019,0,0) ├┤ X ├┤ U1(2.2713) ├──■────────────────┤M├───
     └┬─────────┬─┘                                     │                   ┌─────────┐  │       └─────────────┘     └─┬─┘└┬──────────────────┤└─┬─┘└┬──────────┬┘  │  ┌─┐           └╥┘   
q_1: ─┤ U2(0,π) ├───────────────────────────────────────┼────────────────■──┤ U2(0,0) ├──┼─────────────────────────────■───┤ U3(0.016019,0,0) ├──■───┤ U2(-π,0) ├───┼──┤M├────────────╫────
      └─────────┘  ┌───┐        ┌────────────┐        ┌─┴─┐┌──────────┐┌─┴─┐├─────────┤┌─┴─┐┌───────────────────────┐      └──────────────────┘      └──────────┘ ┌─┴─┐└╥┘┌─────────┐ ║ ┌─┐
q_2: ──────■───────┤ X ├────────┤ U2(π/4,-π) ├────────┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U3(1.5812,1.0494,π/4) ├─────────────────────────────────────────────┤ X ├─╫─┤ U1(π/2) ├─╫─┤M├
         ┌─┴─┐     └─┬─┘┌───────┴────────────┴───────┐└┬─┬┘└──────────┘└───┘└─────────┘└───┘└───────────────────────┘                                             └───┘ ║ └─────────┘ ║ └╥┘
q_3: ────┤ X ├───────■──┤ U3(1.5646,1.5677,-0.65909) ├─┤M├──────────────────────────────────────────────────────────────────────────────────────────────────────────────╫─────────────╫──╫─
         └───┘          └────────────────────────────┘ └╥┘                                                                                                              ║             ║  ║ 
c: 4/═══════════════════════════════════════════════════╩═══════════════════════════════════════════════════════════════════════════════════════════════════════════════╩═════════════╩══╩═
                                                        3                                                                                                               1             0  2

In [6]:
resource_estimate = superstaq.resource_estimate(circuit_random, "ss_unconstrained_simulator")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=18, num_two_qubit_gates=8, depth=16)


# Example on Circuit Compiled on IBMQ Device

In [7]:
circuit2 = qiskit.QuantumCircuit(2)
circuit2.swap(0, 1)
circuit2.draw()

q_0: ─X─
      │ 
q_1: ─X─

In [8]:
out = superstaq.ibmq_compile(circuit2, target="ibmq_brisbane_qpu")
out.circuit.draw(fold=-1)

┌─────────┐┌──────────────┐┌─────────┐ ┌────┐┌───────┐┌─────────────┐┌──────────┐┌────┐┌───────┐┌─────────────┐
q_0: ┤ Rz(π/2) ├┤0             ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(π) ├┤0            ├┤ Rz(3π/2) ├┤ √X ├┤ Rz(π) ├┤0            ├
     └─────────┘│  Acecr(-π/2) │├─────────┴┐├────┤└───────┘│  Acecr(π/2) │├──────────┤└────┘└───────┘│  Acecr(π/2) │
q_1: ───────────┤1             ├┤ Rz(3π/2) ├┤ √X ├─────────┤1            ├┤ Rz(3π/2) ├───────────────┤1            ├
                └──────────────┘└──────────┘└────┘         └─────────────┘└──────────┘               └─────────────┘
q_2: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                    
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                    
q_4: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                    
q_5: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                    
q_6: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [9]:
resource_estimate = superstaq.resource_estimate(circuit2, "ibmq_brisbane_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=10, num_two_qubit_gates=3, depth=10)


# Multiple Circuit Example

In [10]:
# Create List of random circuits (3)
circuits = [
    random_circuit(2, 2, measure=True),
    random_circuit(3, 3, measure=True),
    random_circuit(4, 4, measure=True),
]
circuits = [qiskit.transpile(c, basis_gates=["id", "u1", "u2", "u3", "cx"]) for c in circuits]
for c in circuits:
    print(c)

global phase: π/2
     ┌───────────────────┐┌───┐┌─────────────────┐┌───┐┌─────────────┐┌─┐   
q_0: ┤ U3(π,-2.0177,π/2) ├┤ X ├┤ U3(-1.3811,0,0) ├┤ X ├┤ U1(-2.6947) ├┤M├───
     └───┬────────────┬──┘└─┬─┘└┬────────────────┤└─┬─┘└┬───────────┬┘└╥┘┌─┐
q_1: ────┤ U2(-π/2,0) ├─────■───┤ U3(1.3811,0,0) ├──■───┤ U2(-π,-π) ├──╫─┤M├
         └────────────┘         └────────────────┘      └───────────┘  ║ └╥┘
c: 2/══════════════════════════════════════════════════════════════════╩══╩═
                                                                       0  1 
global phase: 5.7558
       ┌────────────┐                                        »
q_0: ──┤ U1(4.2614) ├─────────────────────■──────────────────»
      ┌┴────────────┴┐ ┌───┐┌──────────┐┌─┴─┐┌─────────┐┌───┐»
q_1: ─┤ U2(π/4,-π/2) ├─┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├»
     ┌┴──────────────┴┐└─┬─┘└──────────┘└───┘└─────────┘└─┬─┘»
q_2: ┤ U3(3.2073,0,0) ├──■────────────────────────────────■──»
     └────────────────┘                     

In [11]:
resource_estimates = superstaq.resource_estimate(circuits, "ss_unconstrained_simulator")
print(resource_estimates)

[ResourceEstimate(num_single_qubit_gates=8, num_two_qubit_gates=2, depth=6), ResourceEstimate(num_single_qubit_gates=11, num_two_qubit_gates=4, depth=10), ResourceEstimate(num_single_qubit_gates=21, num_two_qubit_gates=11, depth=16)]
